In [2]:
%load_ext autoreload
%autoreload 2
import gc
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import timer, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# feim = pd.read_csv('../output/valid_single_feature_tochi.csv', header=None)
feim = pd.read_csv('../output/valid_single_feature.csv', header=None)
feim.columns = ['datetime', 'DT-M', 'feature', 'score']

# feim = feim[feim['datetime']<='20190913_22380']
print(feim['datetime'].max())

feim.set_index('feature', inplace=True)
cnt = feim.groupby('feature')['score'].count()
idx = cnt[cnt==3].index
feim = feim.loc[idx]
may = feim[feim['DT-M']=='2018-5']
may.sort_values(by='score', ascending=False, inplace=True)
feim.loc[may.index]

20190914_12354


,datetime,DT-M,score
feature,,,
523__R_emaildomain_prefix-card1-card3-card4-card5-card6_future_datetime_shift_m1_diff_train,20190914_11355,2018-5,0.935351
523__R_emaildomain_prefix-card1-card3-card4-card5-card6_future_datetime_shift_m1_diff_train,20190914_11355,2018-4,0.948056
523__R_emaildomain_prefix-card1-card3-card4-card5-card6_future_datetime_shift_m1_diff_train,20190914_11355,2018-3,0.936385
522__P_emaildomain_prefix-card4-card5_last_D3_train,20190914_11214,2018-5,0.935274
522__P_emaildomain_prefix-card4-card5_last_D3_train,20190914_11214,2018-4,0.948337
522__P_emaildomain_prefix-card4-card5_last_D3_train,20190914_11214,2018-3,0.934789
606__card2-card3-card5-R_emaildomain_prefix_C8__ProductCD-S_std_train,20190914_12062,2018-5,0.935269
606__card2-card3-card5-R_emaildomain_prefix_C8__ProductCD-S_std_train,20190914_12062,2018-4,0.945875
606__card2-card3-card5-R_emaildomain_prefix_C8__ProductCD-S_std_train,20190914_12062,2018-3,0.935083
